In [ ]:
!wget --user-agent "Mozilla" "https://arxiv.org/pdf/2307.09288.pdf" -O "llama2.pdf"

In [ ]:
!pip install openai pyMuPDF

In [ ]:
import os
import openai
import fitz
from PIL import Image

In [ ]:
pdf_file = "llama2.pdf"

# Split the base name and extension
output_directory_path, _ = os.path.splitext(pdf_file)

if not os.path.exists(output_directory_path):
    os.makedirs(output_directory_path)

image_paths = []

# Open the PDF file
pdf_document = fitz.open(pdf_file)

# Iterate through each page and convert to an image
for page_number in range(pdf_document.page_count):
    # Get the page
    page = pdf_document[page_number]

    # Convert the page to an image
    pix = page.get_pixmap()

    # Create a Pillow Image object from the pixmap
    image = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)

    # Save the image
    image.save(f"./{output_directory_path}/page_{page_number + 1}.png")

# Close the PDF file
pdf_document.close()

In [ ]:
# convert images to markdown

In [ ]:
import base64
import requests

# OpenAI API key


# Function to encode the image
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

def extract_content_from_image(image_path, api_key):
  # Path to your image
  #image_path = "./llama2/page_2.png"

  # Getting the base64 string
  base64_image = encode_image(image_path)

  headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
  }

  payload = {
    "model": "gpt-4o-mini",
    "messages": [
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "extract the contents of the image as markdown"
        },
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}"
          }
        }
      ]
    }
    ],
    "max_tokens": 4000
  }

  response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)
  r = response.json()
  return r

  #print(response.json())

{'id': 'chatcmpl-9u2A3GrLzwjylNl2sLY4oPpERxHII', 'object': 'chat.completion', 'created': 1723141703, 'model': 'gpt-4o-mini-2024-07-18', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': '```markdown\n# Contents\n\n1. Introduction ......................................................... 3\n\n2. Pretraining .......................................................... 5  \n   2.1 Pretraining Data ................................................. 5  \n   2.2 Training Details ................................................. 5  \n   2.3 LLAMA 2 Pretrained Model Evaluation ............................... 7  \n\n3. Fine-tuning ........................................................... 8  \n   3.1 Supervised Fine-Tuning (SFT) ..................................... 9  \n   3.2 Reinforcement Learning with Human Feedback (RLHF) .................. 9  \n   3.3 System Message for Multi-Turn Consistency ........................ 16  \n   3.4 RLHF Results .............................

In [ ]:

import os

def list_png_files(folder_path):
    # Get the list of all files in the specified folder
    files = os.listdir(folder_path)
            
    # Filter the list to include only .png files
    png_files = [os.path.join(folder_path, file) for file in files if file.endswith('.png')]
                        
    return png_files

image_paths=list_png_files(output_directory_path)

api_key = ''

for image_path in image_paths:
    r = extract_content_from_image(image_path, api_key)
    md = r['choices'][0]['message']['content']
    with open(image_path.replace('.png', '.md'), 'w') as f:
        f.write(md)     
    